In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import zipfile

train_df_promt = pd.read_csv('prompts_train.csv')
train_df = pd.read_csv('summaries_train.csv')
test_df_promt = pd.read_csv('prompts_test.csv')
test_df = pd.read_csv('summaries_test.csv')
print(f'Size train set: {train_df.shape} \nSize test set: {test_df.shape}')
print(f'Size train promts: {train_df_promt.shape} \nSize test promts: {test_df_promt.shape}')


Size train set: (7165, 5) 
Size test set: (4, 3)
Size train promts: (4, 4) 
Size test promts: (2, 4)


In [ ]:
import re
import string

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import scipy.stats as stats
import statsmodels.api as sm

In [ ]:
def truncate_text(text, max_length=20):
    """Truncates the string to the specified number of characters"""
    if len(text) > max_length:
        return text[:max_length] + '...'
    return text

train_sample_promt = train_df_promt.head(4).copy()
test_sample_promt = test_df_promt.head(2).copy()
train_sample = train_df.head(4).copy()
test_sample = test_df.head(2).copy()

train_sample_promt.loc[:, 'prompt_text'] = train_sample_promt['prompt_text'].apply(truncate_text)
train_sample_promt.loc[:, 'prompt_question'] = train_sample_promt['prompt_question'].apply(truncate_text)
test_sample_promt.loc[:, 'prompt_text'] = test_sample_promt['prompt_text'].apply(truncate_text)
train_sample.loc[:, 'text'] = train_sample['text'].apply(truncate_text)

In [ ]:
train_sample.head()

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was a...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class wa...,-0.210614,-0.471415


In [ ]:
train_df_promt

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [ ]:
df_train = train_df.merge(train_df_promt, on='prompt_id')

In [ ]:
def columns_is_null(df: pd.DataFrame) -> pd.DataFrame():
    """Returns a dataframe with the characteristics of missing, empty, and unique values for each
    the dataframe column
    """
    values_null = {}

    for column_name in df.columns:
        count_null = df[column_name].isnull().sum()
        count_zero =  len(df[df[column_name] == 0])
        сount_unique = len(df[column_name].unique())
        values_null[column_name] = {"count_null" : count_null,
                                    "count_zerro": count_zero,
                                    "percentage_null_values": round((count_null/df.shape[0])*100, 3),
                                    "percentage_zero_values": round((count_zero/df.shape[0])*100, 3),
                                    "count_uniq": сount_unique}


    return pd.DataFrame(values_null)

In [ ]:
columns_is_null(df_train)

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text
count_null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
count_zerro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
percentage_null_values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
percentage_zero_values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
count_uniq,7165.0,4.0,7165.0,1134.0,1134.0,4.0,4.0,4.0


# TexT PreProcessing¶


In [ ]:
df_train

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
1,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
2,0095993991fe,814d6b,The third wave only started as an experiment w...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,00c20c6ddd23,814d6b,The experimen was orginally about how even whe...,0.567975,0.969062,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
4,00d40ad10dc9,814d6b,The third wave developed so quickly due to the...,-0.910596,-0.081769,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
...,...,...,...,...,...,...,...,...
7160,fef3e85236e5,39c16e,"It has to be made on a complex storyline, with...",-0.981265,-1.548900,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
7161,ff0f65eecf02,39c16e,Aristotle descirbes an ideal tradgedy as being...,-0.511077,-1.589115,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
7162,ff186473ea0a,39c16e,A tragedy should have a complex plan not a sim...,-0.834946,-0.593749,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
7163,ff5e9e6068da,39c16e,Aristotle believed that the ideal tradegy shou...,-0.157460,-0.165811,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
def clean_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление знаков препинания, специальных символов и чисел
    text = re.sub(r'[^a-z\s]', '', text)
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

In [ ]:
stop_words = {
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",
    "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
    "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
    "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
    "the", "and", "but", "if", "or", "because", "as", "until", "while", "of",
    "at", "by", "for", "with", "about", "against", "between", "into", "through",
    "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then",
    "once", "here", "there", "when", "where", "why", "how", "all", "any",
    "both", "each", "few", "more", "most", "other", "some", "such", "no",
    "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s",
    "t", "can", "will", "just", "don", "should", "now"
}

def remove_stopwords(tokens):
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

In [ ]:
from nltk.stem import WordNetLemmatizer

def lemmatize_text(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

In [ ]:
df_train['text'] = df_train['text'].apply(clean_text)

In [ ]:
df_train['text'].head()

,text
0,the third wave was an experimentto see how peo...
1,the third wave developed rapidly because the s...
2,the third wave only started as an experiment w...
3,the experimen was orginally about how even whe...
4,the third wave developed so quickly due to the...


In [ ]:
df_train['text'] = df_train['text'].apply(tokenize_text)
df_train['text'] = df_train['text'].apply(lemmatize_text)

In [ ]:
df_train['text'].head()

,text
0,"[the, third, wave, wa, an, experimentto, see, ..."
1,"[the, third, wave, developed, rapidly, because..."
2,"[the, third, wave, only, started, a, an, exper..."
3,"[the, experimen, wa, orginally, about, how, ev..."
4,"[the, third, wave, developed, so, quickly, due..."


In [ ]:
df_train['text'] = df_train['text'].apply(remove_stopwords)

In [ ]:
X = df_train['text']  # Признаки: текст
y_content = df_train['content']  # Целевая переменная 1: content
y_wording = df_train['wording']  # Целевая переменная 2: wording

df_train = df_train.drop('content', axis=1)
df_train = df_train.drop('wording', axis=1)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train_content, y_test_content, y_train_wording, y_test_wording = train_test_split(
    X, y_content, y_wording, test_size=0.2, random_state=42)

In [ ]:
df_train['text'].head()

,text
0,"[third, wave, wa, experimentto, see, people, r..."
1,"[third, wave, developed, rapidly, student, gen..."
2,"[third, wave, started, experiment, within, cla..."
3,"[experimen, wa, orginally, even, terrible, thn..."
4,"[third, wave, developed, quickly, due, student..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

vectorizer = TfidfVectorizer(max_features=5000)
# Если X_train содержит списки токенов, нужно объединить их обратно в строки
X_train = [' '.join(text) if isinstance(text, list) else text for text in X_train]
X_test = [' '.join(text) if isinstance(text, list) else text for text in X_test]

# Теперь можно преобразовать данные в TF-IDF
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

model_content = RandomForestRegressor(random_state=42)
model_content.fit(X_train_tfidf, y_train_content)

model_wording = RandomForestRegressor(random_state=42)
model_wording.fit(X_train_tfidf, y_train_wording)

y_pred_content = model_content.predict(X_test_tfidf)
y_pred_wording = model_wording.predict(X_test_tfidf)

mse_content = mean_squared_error(y_test_content, y_pred_content)
mse_wording = mean_squared_error(y_test_wording, y_pred_wording)

print(f'MSE for content: {mse_content}')
print(f'MSE for wording: {mse_wording}')

# with lemmitazing
# MSE for content: 0.33849508486752766
# MSE for wording: 0.5032854969868331

# without lemmitazing
# MSE for content: 0.35078845976856765
# MSE for wording: 0.49522673371806175

MSE for content: 0.3308614791221092
MSE for wording: 0.5007761718568188


In [ ]:
def calculate_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse


mcrmse_score_content = calculate_rmse(y_test_content, y_pred_content)
mcrmse_score_wording = calculate_rmse(y_test_wording, y_pred_wording)

print(f"MCRMSE = {np.mean([mcrmse_score_content, mcrmse_score_wording])}")
# MCRMSE without lemmitazing = 0.6479987258666542
# MCRMSE with lemmitazing = 0.6414305010009269

MCRMSE = 0.6414305010009269
